<strong> 데이터 세트 구축 <br></strong>
train : 2018년 1월 1일 ~ 2020년 12월 31일 <br>
val : 2021년 1월 1일 ~ 2021년 6월 30일 <br>
test : 2021년 7월 1일 ~ 2021년 12월 31일 <br>
<strong>데이터셋 생성조건</strong><br> 
- 거래대금 : 1조,1000억, 100억, 10억,1억
- 거래일 : 10일
- next_change : (0.05, 0.02)

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime
import numpy as np
import pickle
import pymysql
import warnings
warnings.filterwarnings(action = 'ignore')

# 데이터셋 구축 & 파일저장 함수

In [2]:
def make_data_train(trading, days, label): 
    # 코드 리스트 
   
    IF=open("../../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    
    # train

    lst_train = []

    
    # database
    # con = pymysql.connect()

    cursor = con.cursor()
 
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE (Date BETWEEN '2018-01-01' AND '2020-12-31') AND (Low not in (0))
                    '''.format(code)
        stock = pd.read_sql(sql = sql_query, con = con)
        stock.dropna(inplace=True) # nan 제거 
        lst_stock = stock.values.tolist()  


        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            # 거래대금, 이동평균선 범위 조건
            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                lst_result = []
                
                Ddays_date = lst_stock[i-(days-1):i+1]
                                           
                for row2 in Ddays_date:
                    Open,High,Low,Close,Volume = row2[1:6]
                    Trading = Close * Volume
                    lst_result += [Open,High,Low,Close,Trading]
                y = int(row[6] >= label)
                target = float(row[6])
                
                data = ','.join(map(str,lst_result))
                

                lst_train.append([code,date,data,y])

    return np.array(lst_train)

In [3]:
def make_data_val(trading, days, label): 
    # 코드 리스트 
   
    IF=open("../../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    


    lst_val = []

    # database
    # con = pymysql.connect()
    cursor = con.cursor()
 
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE (Date BETWEEN '2021-01-01' AND '2021-06-30') AND (Low not in (0))
                    '''.format(code)
        stock = pd.read_sql(sql = sql_query, con = con)
        stock.dropna(inplace=True) # nan 제거 
        lst_stock = stock.values.tolist()  


        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            # 거래대금, 이동평균선 범위 조건
            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                lst_result = []
                
                Ddays_date = lst_stock[i-(days-1):i+1]
                                           
                for row2 in Ddays_date:
                    Open,High,Low,Close,Volume = row2[1:6]
                    Trading = Close * Volume
                    lst_result += [Open,High,Low,Close,Trading]
                y = int(row[6] >= label)
                target = float(row[6])
                data = ','.join(map(str,lst_result))
                
     
                lst_val.append([code,date,data,y])

    return np.array(lst_val)

In [4]:
def make_data_test(trading, days, label): 
    # 코드 리스트 
   
    IF=open("../../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    

    lst_test = []
    
    # database
    # con = pymysql.connect()

    cursor = con.cursor()
 
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE (Date BETWEEN '2021-07-01' AND '2021-12-31') AND (Low not in (0))
                    '''.format(code)
        stock = pd.read_sql(sql = sql_query, con = con)
        stock.dropna(inplace=True) # nan 제거 
        lst_stock = stock.values.tolist()  


        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            # 거래대금, 이동평균선 범위 조건
            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                lst_result = []
                
                Ddays_date = lst_stock[i-(days-1):i+1]
                                           
                for row2 in Ddays_date:
                    Open,High,Low,Close,Volume = row2[1:6]
                    Trading = Close * Volume
                    lst_result += [Open,High,Low,Close,Trading]
                y = int(row[6] >= label)
                target = float(row[6])
                data = ','.join(map(str,lst_result))
                

                lst_test.append([code,date,data,y])

    return np.array(lst_test)

In [14]:
def dump_file(TYPE, trading, days,target,data):
    with open('../../data/{}_{}_{}_{}.pickle'.format(TYPE,trading, days,target), 'wb') as f:
        pickle.dump(data, f)

# Change 0.02
## 거래대금 1조 
함수보고 리턴값 바꾸기

In [13]:
train_data = make_data_train(10000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:08<00:00, 22.84it/s]


In [14]:
val_data = make_data_val(10000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:29<00:00, 53.81it/s]


In [15]:
test_data= make_data_test(10000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:28<00:00, 54.36it/s]


In [17]:
train_data.shape, val_data.shape, test_data.shape

((350, 4), (187, 4), (82, 4))

In [18]:
dump_file('train',10000, 10, 0.02,train_data)
dump_file('val',10000, 10, 0.02,val_data)
dump_file('test',10000, 10, 0.02,test_data)


## 거래대금 1천억 

In [19]:
train_data = make_data_train(1000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:11<00:00, 21.87it/s]


In [20]:
val_data = make_data_val(1000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:32<00:00, 48.38it/s]


In [21]:
test_data= make_data_test(1000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:34<00:00, 44.94it/s]


In [22]:
train_data.shape, val_data.shape, test_data.shape

((11975, 4), (4467, 4), (3585, 4))

In [23]:
dump_file('train',1000, 10, 0.02,train_data)
dump_file('val',1000, 10, 0.02,val_data)
dump_file('test',1000, 10, 0.02,test_data)

## 거래대금 100억 

In [24]:
train_data = make_data_train(100, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:14<00:00, 20.98it/s]


In [25]:
val_data = make_data_val(100, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:32<00:00, 48.71it/s]


In [26]:
test_data= make_data_test(100, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:34<00:00, 45.06it/s]


In [29]:
train_data.shape, val_data.shape, test_data.shape

((133796, 4), (35297, 4), (28895, 4))

In [30]:
dump_file('train',100, 10, 0.02,train_data)
dump_file('val',100, 10, 0.02,val_data)
dump_file('test',100, 10, 0.02,test_data)

## 거래대금 10억

In [31]:
train_data = make_data_train(10, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:54<00:00, 13.64it/s]


In [32]:
val_data = make_data_val(10, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:53<00:00, 29.43it/s]


In [33]:
test_data= make_data_test(10, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:41<00:00, 38.01it/s]


In [34]:
train_data.shape, val_data.shape, test_data.shape

((513097, 4), (120470, 4), (101004, 4))

In [35]:
dump_file('train',10, 10, 0.02,train_data)
dump_file('val',10, 10, 0.02,val_data)
dump_file('test',10, 10, 0.02,test_data)

# Scaling

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
scaler = MinMaxScaler()

In [4]:
def make_data_scaling_train(trading, days, label): 

    # 코드 리스트 
    IF=open("../../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    


    col_scaling = ['Open', 'High', 'Low', 'Close', 'Trading']  
    # trainX, trainY
    lst_train = []
    
    # database
    con = pymysql.connect(
        host = 'ahnbi2.suwon.ac.kr', 
        port = 3306, 
        user = 'stock_user', 
        passwd = 'bigdata', 
        db = 'stock_db', 
        charset = 'utf8'
    )

    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE (Date BETWEEN '2018-01-01' AND '2020-12-31') AND (Low not in (0))
                    '''.format(code)

        stock = pd.read_sql(sql=sql_query, con= con)
        stock.dropna(inplace=True) # nan 제거 
        lst_stock = stock.values.tolist()
        
        stock['Trading'] = stock['Close'] * stock['Volume'] # 거래대금 컬럼 추가 
        scaler = MinMaxScaler()
        scale_stock = scaler.fit_transform(stock[col_scaling])
        df_stock_scale = pd.DataFrame(scale_stock, columns = ['Open', 'High', 'Low', 'Close', 'Trading'])
        lst_stock_scale = df_stock_scale.values.tolist()
        
        
        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            # 거래대금, 이동평균선 범위 조건
            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                result=[]
                for row2 in lst_stock_scale[i-(days-1):i+1]:  
                    result += row2 # scaling                 
                y = int(row[6]>=label) # next change 사용    
                
                data = ','.join(map(str,result))
                
                lst_train.append([code,date,data,y])

    return np.array(lst_train)

In [3]:
def make_data_scaling_val(trading, days, label): 
    # 코드 리스트 
   
    IF=open("../../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    

    lst_val = []

     # database
    con = pymysql.connect(
        host = 'ahnbi2.suwon.ac.kr', 
        port = 3306, 
        user = 'stock_user', 
        passwd = 'bigdata', 
        db = 'stock_db', 
        charset = 'utf8'
    )

    cursor = con.cursor()
    
    col_scaling = ['Open', 'High', 'Low', 'Close', 'Trading']   

       
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE (Date BETWEEN '2021-01-01' AND '2021-06-30') AND (Low not in (0))
                    '''.format(code)

        stock = pd.read_sql(sql=sql_query, con= con)
        stock.dropna(inplace=True) # nan 제거 
        lst_stock = stock.values.tolist()
        
        stock['Trading'] = stock['Close'] * stock['Volume'] # 거래대금 컬럼 추가 
        scaler = MinMaxScaler()
        scale_stock = scaler.fit_transform(stock[col_scaling])
        df_stock_scale = pd.DataFrame(scale_stock, columns = ['Open', 'High', 'Low', 'Close', 'Trading'])
        lst_stock_scale = df_stock_scale.values.tolist()
        
        
        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            # 거래대금, 이동평균선 범위 조건
            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                result=[]
                for row2 in lst_stock_scale[i-(days-1):i+1]:  
                    result += row2 # scaling                 
                y = int(row[6]>=label) # next change 사용    
                
                data = ','.join(map(str,result))     
                lst_val.append([code,date,data,y])

    return np.array(lst_val)

In [2]:
def make_data_scaling_test(trading, days, label): 
    # 코드 리스트 
   
    IF=open("../../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    
    lst_test = []
    
    # database
    con = pymysql.connect(
        host = 'ahnbi2.suwon.ac.kr', 
        port = 3306, 
        user = 'stock_user', 
        passwd = 'bigdata', 
        db = 'stock_db', 
        charset = 'utf8'
    )
    
    col_scaling = ['Open', 'High', 'Low', 'Close', 'Trading']   

    col_no_scaling = ['Change']
       
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE (Date BETWEEN '2021-07-01' AND '2021-12-31') AND (Low not in (0))
                    '''.format(code)

        stock = pd.read_sql(sql=sql_query, con= con)
        stock.dropna(inplace=True) # nan 제거 
        lst_stock = stock.values.tolist()
        
        stock['Trading'] = stock['Close'] * stock['Volume'] # 거래대금 컬럼 추가 
        scaler = MinMaxScaler()
        scale_stock = scaler.fit_transform(stock[col_scaling])
        df_stock_scale = pd.DataFrame(scale_stock, columns = ['Open', 'High', 'Low', 'Close', 'Trading'])
        lst_stock_scale = df_stock_scale.values.tolist()
        
        
        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            # 거래대금, 이동평균선 범위 조건
            if (close * volume >= trading*100000000) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                result=[]
                for row2 in lst_stock_scale[i-(days-1):i+1]:  
                    result += row2 # scaling                 
                y = int(row[6]>=label) # next change 사용    
                
                data = ','.join(map(str,result))

                lst_test.append([code,date,data,y])

    return np.array(lst_test)

In [8]:
train = make_data_scaling_train(100, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:49<00:00, 14.22it/s]


In [133]:
val = make_data_scaling_val(100, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:39<00:00, 39.62it/s]


In [134]:
test = make_data_scaling_test(100, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:40<00:00, 38.64it/s]


In [135]:
train.shape, val.shape, test.shape

((133796, 4), (35297, 4), (28895, 4))

In [136]:
dump_file('scaling_train',100, 10, 0.02,train)
dump_file('scaling_val',100, 10, 0.02,val)
dump_file('scaling_test',100, 10, 0.02,test)

## 거래대금 100억 & 임계값 설정 
### change 0.05

In [9]:
train = make_data_scaling_train(100, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:01<00:00, 25.57it/s]


In [10]:
val = make_data_scaling_val(100, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:29<00:00, 52.92it/s]


In [11]:
test = make_data_scaling_test(100, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:27<00:00, 57.75it/s]


In [12]:
train.shape, val.shape, test.shape

((133796, 4), (35297, 4), (28895, 4))

In [15]:
dump_file('scaling_train',100, 10, 0.05,train)
dump_file('scaling_val',100, 10, 0.05,val)
dump_file('scaling_test',100, 10, 0.05,test)

# 거래일 설정
## 5일

In [16]:
train_data = make_data_scaling_train(100, 5, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:04<00:00, 24.13it/s]


In [17]:
val_data = make_data_scaling_val(100, 5, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:30<00:00, 51.67it/s]


In [18]:
test_data= make_data_scaling_test(100, 5, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:27<00:00, 57.22it/s]


In [19]:
dump_file('scaling_train',100, 5, 0.02,train_data)
dump_file('scaling_val',100, 5, 0.02,val_data)
dump_file('scaling_test',100, 5, 0.02,test_data)

## 20일

In [20]:
train_data = make_data_scaling_train(100, 20, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:06<00:00, 23.50it/s]


In [21]:
val_data = make_data_scaling_val(100, 20, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:31<00:00, 49.21it/s]


In [22]:
test_data= make_data_scaling_test(100, 20, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:28<00:00, 54.61it/s]


In [23]:
dump_file('scaling_train',100, 20, 0.02,train_data)
dump_file('scaling_val',100, 20, 0.02,val_data)
dump_file('scaling_test',100, 20, 0.02,test_data)